# Example: structure factors

In this notebook, we show how to conduct size distribution inversion with structure factors for high-concentration systems. In the next version of `ffsas`, we will make this much easier with a new user API.

In [ ]:
from pathlib import Path
import pickle

import matplotlib.pyplot as plt
import numpy as np
import torch
from scipy import optimize, interpolate

import ffsas
from ffsas.models import Sphere
from ffsas.system import SASGreensSystem

# avoid an OMP error on MacOS (nothing to do with ffsas)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
# reproduce figures in the paper 
reproduce_paper_fig = True
if reproduce_paper_fig:
    # this will trigger an error if latex is not installed
    plt.rcParams.update({
        "text.usetex": True,
        "text.latex.preamble": r'\usepackage{bm,upgreek}',
        "font.family": "sans-serif",
        "font.serif": ["Times"]})
    # figure dir
    paper_fig_dir = Path('../paper_figs')
    Path(paper_fig_dir).mkdir(parents=True, exist_ok=True)

# Core functions

In [ ]:
# compute the structure factor S given q, effective radius and volume fraction
# assuming the "hard sphere" model
# adapted from hardsphere.py downloaded from 
# https://www.sasview.org/docs/user/models/hardsphere.html
def compute_S_hard_sphere(q, r_eff, vol_frac):
    D = (1.0 / (1.0 - vol_frac)) ** 2
    A = ((1. + 2. * vol_frac) * D) ** 2
    X = np.abs(q * r_eff * 2.0)
    X2 = X * X
    B = -6. * vol_frac * ((1.0 + 0.5 * vol_frac) * D) ** 2
    G = 0.5 * vol_frac * A
    X4 = X2 * X2
    S = np.sin(X)
    C = np.cos(X)
    FF = ((G * ((4. * X2 - 24.) * X * S - (
        X4 - 12. * X2 + 24.) * C + 24.) / X2 + B * (
        2. * X * S - (X2 - 2.) * C - 2.)) / X + A * (S - X * C)) / X
    SF = 1. / (1. + 24. * vol_frac * FF / X2)
    return SF

# compute intensity
# https://www.sasview.org/docs/user/qtgui/Perspectives/Fitting/fitting_sq.html
def compute_I(G, w, q, r_vec_or_r_eff, vol_frac, xi, b, beta=None):
    # form factor
    P = np.dot(G, w)
    # structure factor
    if hasattr(r_vec_or_r_eff, "__len__"):
        r_eff = (w * r_vec_or_r_eff ** 3).sum() ** (1 / 3)
    else:
        r_eff = r_vec_or_r_eff
    S = compute_S_hard_sphere(q, r_eff, vol_frac)
    # scale and background
    if beta is None:
        I = xi * vol_frac * P * S + b
    else:
        I = xi * vol_frac * P * (1. + beta * (S - 1.)) + b
    return I
    
# compute chi2
def compute_chi2(G, w, q, r_vec_or_r_eff, vol_frac, xi, b, mu, sigma, beta=None):
    I = compute_I(G, w, q, r_vec_or_r_eff, vol_frac, xi, b, beta) 
    eps = (I - mu) / sigma
    chi2 = (eps * eps).sum()
    return chi2

In [ ]:
# test structure factor using default of SasView
# https://www.sasview.org/docs/user/models/hardsphere.html
q = 10 ** np.linspace(-3, 0, 200)
S = compute_S_hard_sphere(q, r_eff=50, vol_frac=.2)
plt.figure(dpi=100)
plt.plot(q, S)
plt.xscale('log')
plt.xlabel('Scattering vector, $q$ (\AA$^{-1}$)')
plt.ylabel('Structure factor, $S$')
plt.show()

# Data

In [ ]:
# read data
fname = 'S49_Ludox6_1pct.dat'
data = np.loadtxt('data/' + fname, skiprows=1)

# q vector
q = data[:, 0]

# intensity mean
mu = data[:, 1]

# intensity stddev
sigma = data[:, 2]

# truncate at low-q
tr = 60
q_tr = q[tr:]
mu_tr = mu[tr:]
sigma_tr = sigma[tr:]

# plot
plt.figure(dpi=100)
plt.errorbar(q, mu, yerr=sigma, ecolor='g')
plt.axvline(q_tr[0], c='r')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Scattering vector, $q$ (\AA$^{-1}$)')
plt.ylabel('Intensity, $I$ ($\mathrm{cm}^{-1}$)')
plt.show()

Compute the Green's tensor to compute form factor:

In [ ]:
# specify radii
r = 10 ** np.linspace(0, 2.5, 1000)

# compute the Green's tensor
G = Sphere.compute_G_mini_batch([torch.tensor(q_tr)], 
                                {'r': torch.tensor(r)}, 
                                {'drho': 1.}, log_screen=False)

Here we do a "fake" inversion (only with one iteration) to find the auto-scaling factors for `xi` and `b`:

In [ ]:
# build G-system
g_sys = SASGreensSystem(G, Sphere.get_par_keys_G(), log_screen=False)

# fake inversion
results = g_sys.solve_inverse(mu_tr, sigma_tr, maxiter=1)

# auto scales
xi_mag = g_sys._xi_mag
b_mag = g_sys._b_mag
print(f'Order of magnitude of xi: {xi_mag}')
print(f'Order of magnitude of b: {b_mag}')

# scale the data to pre-condition the inverse problem
G_scaled = (G * xi_mag / b_mag).numpy()
mu_tr_scaled =  mu_tr / b_mag
sigma_tr_scaled = sigma_tr / b_mag

# Inversion

### 1. Make an initial guess


In [ ]:
# whether to use independent r_eff when computing S
independent_r_eff = True

# whether to consider beta when computing S
consider_beta = False

In [ ]:
# uniform weights
w0 = np.ones_like(r) / len(r)
xi_scaled0 = 100
b_scaled0 = 2.5
vol_frac0 = 0.05
r_eff0 = 180
beta0 = np.ones_like(q_tr)

# flattend X0
X0 = np.concatenate((np.sqrt(w0), [xi_scaled0, b_scaled0, vol_frac0]))
if independent_r_eff:
    X0 = np.concatenate((X0, [r_eff0]))
if consider_beta:
    X0 = np.concatenate((X0, np.sqrt(beta0)))

In [ ]:
# intensity at initial guess
I_pred0 = compute_I(G=G, w=w0, 
                    q=q_tr, r_vec_or_r_eff=r_eff0, vol_frac=vol_frac0, 
                    xi=xi_scaled0 * xi_mag, b=b_scaled0 * b_mag, beta=beta0)
plt.figure(dpi=100)
plt.plot(q, mu)
plt.plot(q_tr, I_pred0)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Scattering vector, $q$ (\AA$^{-1}$)')
plt.ylabel('Intensity, $I$ ($\mathrm{cm}^{-1}$)')
plt.show()

### 2. Define objective function and constraints for the NLP

In [ ]:
# objective function
def obj_func(X):
    # extract variables from X
    w, xi_scaled, b_scaled, vol_frac = X[:len(r)] ** 2, X[len(r)], X[len(r) + 1], X[len(r) + 2]
    if independent_r_eff:
        r_vec_or_r_eff = X[len(r) + 3]
    else:
        r_vec_or_r_eff = r
    if consider_beta:
        beta = (X[-len(q_tr):]) ** 2
    else:
        beta = None
    # compute chi2
    return compute_chi2(G=G_scaled, w=w, 
                        q=q_tr, r_vec_or_r_eff=r_vec_or_r_eff, vol_frac=vol_frac, 
                        xi=xi_scaled, b=b_scaled, 
                        mu=mu_tr_scaled, sigma=sigma_tr_scaled, 
                        beta=beta)

# constraint function
def constr_s2_func(X):
    s = X[:len(r)]
    err = np.dot(s, s) - 1.
    return np.array([err])

# equility constraint: s.s - 1 == 0
s2 = optimize.NonlinearConstraint(constr_s2_func, 0., 0.)

### 3. Solve the NLP with Trust Region

In [ ]:
# solve inverse
maxiter = 500
res = optimize.minimize(
    obj_func, X0,
    method='trust-constr',
    constraints=(s2,),
    options={'maxiter': maxiter, 'verbose': 0})

In [ ]:
# extract results
X = res['x']
w_MLE, xi_scaled_MLE, b_scaled_MLE, vol_frac_MLE = \
X[:len(r)] ** 2, X[len(r)], X[len(r) + 1], X[len(r) + 2]

if independent_r_eff:
    r_eff_MLE = X[len(r) + 3]
else:
    r_eff_MLE = (w_MLE * r ** 3).sum() ** (1 / 3)
if consider_beta:
    beta_MLE = (X[-len(q_tr):]) ** 2
else:
    beta_MLE = None

### 4. Visualize and save results

In [ ]:
print(f'Inverted volume fraction: {vol_frac_MLE}')
print(f'Inverted effective radius: {r_eff_MLE}')
print(f'Inverted xi: {xi_scaled_MLE * xi_mag}')
print(f'Inverted b: {b_scaled_MLE * b_mag}')

# radius distribution
plt.figure(dpi=100)
plt.plot(r, w_MLE)
plt.xlabel(r'Radius, $r$ (\AA)')
plt.ylabel(r'Weight, $w$ (\%)')
plt.show()

# intensity
I_pred = compute_I(G=G, w=w_MLE, 
                   q=q_tr, r_vec_or_r_eff=r_eff_MLE, vol_frac=vol_frac_MLE, 
                   xi=xi_scaled_MLE * xi_mag, b=b_scaled_MLE * b_mag, beta=beta_MLE)
plt.figure(dpi=100)
plt.plot(q, mu)
plt.plot(q_tr, I_pred)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Scattering vector, $q$ (\AA$^{-1}$)')
plt.ylabel('Intensity, $I$ ($\mathrm{cm}^{-1}$)')
plt.show()

In [ ]:
# save
result_dict = {
    'q': q, 'mu': mu, 'sigma': sigma, 'tr': tr, 'r': r, 
    'xi_scaled0': xi_scaled0,
    'b_scaled0': b_scaled0,
    'vol_frac0': vol_frac0,
    'r_eff0': r_eff0,
    'maxiter': maxiter,
    'w_MLE': w_MLE, 
    'xi_MLE': xi_scaled_MLE * xi_mag, 
    'b_MLE': b_scaled_MLE * b_mag, 
    'vol_frac_MLE': vol_frac_MLE,
    'r_eff_MLE': r_eff_MLE,
    'beta_MLE': beta_MLE,
    'I_pred': I_pred}

with open('results/' + fname.replace('.dat', '.pkl'), 'wb') as f:
        pickle.dump(result_dict, f)

# Figures in paper

In [ ]:
fnames = ['S35_Ludox1_40pct', 'S40_Ludox2_30pct', 'S41_Ludox3_20pct', 
          'S42_Ludox4_10pct', 'S47_Ludox5_5pct', 'S49_Ludox6_1pct']

# read results
res_all = {}
for fn in fnames:
    with open('results/' + f'{fn}.pkl', 'rb') as f:
        res = pickle.load(f)
    res_all[fn] = res

In [ ]:
# find Gaussian approximateion
r_uniform = np.linspace(1, 10 ** 2.5, 10000)
gaussians = []
for i, (fn, res) in enumerate(res_all.items()):
    w = res['w_MLE']
    w_uniform = interpolate.interp1d(r, w)(r_uniform)
    w_uniform /= w_uniform.sum()
    i_top = np.argmax(w_uniform)
    r_top = r_uniform[i_top]
    for i in range(1, 2000):
        area = w_uniform[i_top - i:i_top + i + 1].sum()
        if area > 0.68:
            break
    r_sigma = i * (10 ** 2.5 - 1) / 5000
    gaussians.append((r_top, r_sigma))

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
markers = ['s', 'o', 'd', 'h', '^', 'v']
plt.rcParams.update({'font.size': 10})
plt.rcParams.update({'legend.fontsize': 9.5})
plt.rcParams.update({'axes.titlesize': 10})

fig=plt.figure(dpi=200, figsize=(8, 3.5))

lines = []
labels = []
for i, (fn, res) in enumerate(res_all.items()):
    # data
    pe = plt.errorbar(res['q'][res['tr']:], res['mu'][res['tr']:], 
                      yerr=res['sigma'][res['tr']:] * 5, c=colors[i], ecolor=colors[i], capsize=1,
                      lw=.5, fmt=markers[i], markersize=4, markerfacecolor='none', markeredgewidth=.5,
                      zorder=-i * 10, alpha=.7)
    # fit
    pf = plt.plot(res['q'][res['tr']:], res['I_pred'], c=colors[i], lw=1.3, zorder=-i)
    lines.append((pe, pf[0]))
    labels.append(r"%s: $V_f$=%d%s, $r_\mathrm{eff}$=%d \AA, $w\approx\mathcal{N}(%d, %d^2)$" % (
        fn.replace('_', '-').replace(f'Ludox{i + 1}-', ''), 
        res['vol_frac_MLE'] * 100, '\%', res['r_eff_MLE'],
        gaussians[i][0], gaussians[i][1]))

plt.xlim(1e-3, 2.2e-1)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Scattering vector, $q$ (\AA$^{-1}$)')
plt.ylabel('Intensity, $I$ ($\mathrm{cm}^{-1}$)')
plt.legend(lines, labels, facecolor='whitesmoke')

# save for paper
if reproduce_paper_fig:
    plt.savefig(paper_fig_dir / 'ludox.pdf', bbox_inches='tight', 
                facecolor='w', pad_inches=.05)
plt.show()